У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import OrdinalEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [16]:
cust_df = pd.read_csv(r'D:\ml_course\Модуль 2. Навчання з учителем\dataframes\customer_segmentation_train.csv')

In [17]:
cust_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [18]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [19]:
cust_df.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [20]:
cust_df['Segmentation'].value_counts()

Segmentation
D    2268
A    1972
C    1970
B    1858
Name: count, dtype: int64

In [21]:
X = cust_df.drop(['Segmentation', 'ID'], axis=1)
y = cust_df['Segmentation']

numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes('object').columns.tolist()

X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())
X[categorical_cols] = X[categorical_cols].fillna(X[categorical_cols].mode().iloc[0])

# One-Hot Encoding не застосовувався на цьому етапі, оскільки для подальшого використання методу SMOTENC
# необхідно зберегти категоріальні ознаки.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (6454, 9)
Test shape: (1614, 9)


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [22]:
ord_enc = OrdinalEncoder()
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
X_train_enc[categorical_cols] = ord_enc.fit_transform(X_train[categorical_cols])
X_test_enc[categorical_cols] = ord_enc.transform(X_test[categorical_cols])
cat_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

smotenc = SMOTENC(categorical_features=cat_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train_enc, y_train)

smote_tomek = SMOTETomek(smote=SMOTENC
        (categorical_features=cat_indices,
        random_state=42), random_state=42)
X_res_tomek, y_res_tomek = smote_tomek.fit_resample(X_train_enc, y_train)

print("Before resampling:")
print(y_train.value_counts())
print("Shape:", X_train.shape)

print("\nAfter SMOTENC:")
print(y_train_smotenc.value_counts())
print("Shape:", X_train_smotenc.shape)

print("\nAfter SMOTE-Tomek:")
print(y_res_tomek.value_counts())
print("Shape:", X_res_tomek.shape)

Before resampling:
Segmentation
D    1814
A    1578
C    1576
B    1486
Name: count, dtype: int64
Shape: (6454, 9)

After SMOTENC:
Segmentation
A    1814
B    1814
C    1814
D    1814
Name: count, dtype: int64
Shape: (7256, 9)

After SMOTE-Tomek:
Segmentation
D    1559
C    1554
B    1511
A    1474
Name: count, dtype: int64
Shape: (6098, 9)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_ohe = ohe.fit_transform(X_train_enc[categorical_cols])
X_test_ohe = ohe.transform(X_test_enc[categorical_cols])

# создаем DataFrame
X_train_ohe = pd.DataFrame(
    X_train_ohe,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train_enc.index
)

X_test_ohe = pd.DataFrame(
    X_test_ohe,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test_enc.index
)

# объединяем с числовыми
X_train_final = pd.concat(
    [X_train_enc.drop(columns=categorical_cols), X_train_ohe],
    axis=1
)

X_test_final = pd.concat(
    [X_test_enc.drop(columns=categorical_cols), X_test_ohe],
    axis=1
)

In [27]:
model_original = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, random_state=42)
)

model_original.fit(X_train_final, y_train)

y_pred_original = model_original.predict(X_test_final)

print("=== ORIGINAL DATA ===")
print(classification_report(y_test, y_pred_original))

=== ORIGINAL DATA ===
              precision    recall  f1-score   support

           A       0.42      0.45      0.43       394
           B       0.41      0.18      0.25       372
           C       0.49      0.61      0.54       394
           D       0.65      0.76      0.70       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.51      0.49      1614



In [25]:
# OHE для SMOTENC данных
X_smotenc_cat = ohe.fit_transform(
    X_train_smotenc[categorical_cols]
)

X_smotenc_cat = pd.DataFrame(
    X_smotenc_cat,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train_smotenc.index
)

X_smotenc_final = pd.concat(
    [X_train_smotenc.drop(columns=categorical_cols), X_smotenc_cat],
    axis=1
)

model_smotenc = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, random_state=42)
)

model_smotenc.fit(X_smotenc_final, y_train_smotenc)

y_pred_smotenc = model_smotenc.predict(X_test_final)

print("=== SMOTENC DATA ===")
print(classification_report(y_test, y_pred_smotenc))

=== SMOTENC DATA ===
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.42      0.26      0.32       372
           C       0.52      0.59      0.55       394
           D       0.67      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [26]:
X_tomek_cat = ohe.fit_transform(
    X_res_tomek[categorical_cols]
)

X_tomek_cat = pd.DataFrame(
    X_tomek_cat,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_res_tomek.index
)

X_tomek_final = pd.concat(
    [X_res_tomek.drop(columns=categorical_cols), X_tomek_cat],
    axis=1
)

model_tomek = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, random_state=42)
)

model_tomek.fit(X_tomek_final, y_res_tomek)

y_pred_tomek = model_tomek.predict(X_test_final)

print("=== SMOTE-TOMEK DATA ===")
print(classification_report(y_test, y_pred_tomek))

=== SMOTE-TOMEK DATA ===
              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.41      0.26      0.32       372
           C       0.52      0.59      0.55       394
           D       0.67      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



Для порівняння моделей було використано macro F1-score, оскільки задача є мультикласовою і дана метрика однаково враховує якість передбачення для кожного класу незалежно від їх розміру.
Істотної різниці між моделями не спостерігається. Це можна пояснити тим, що вихідний датасет вже був відносно збалансованим, тому застосування методів ресемплінгу не призвело до значного покращення якості моделі. Крім того, логістична регресія є лінійною моделлю і може не повністю використовувати додаткові синтетичні зразки для покращення меж класифікації.

Для порівняння моделей було використано macro F1-score, оскільки задача є мультикласовою. Дана метрика однаково враховує якість класифікації для кожного класу незалежно від їх розміру.
Суттєвої різниці між моделями не спостерігається. Це можна пояснити тим, що вихідний датасет вже був відносно збалансованим, тому застосування методів ресемплінгу не призвело до значного покращення якості. Крім того, логістична регресія є лінійною моделлю і може не повністю використовувати додаткові синтетичні зразки для покращення меж класифікації.